In [1]:
import openpyxl as pyxl
import pandas as pd
import numpy as np
from gspread.utils import a1_to_rowcol, rowcol_to_a1

from excel_file_wrapper import *

import re
from itertools import product, chain
# we only need the function datetime.datetime.now; we can now reference it as dt.now
from datetime import datetime as dt

import sqlite3 as sql

In [2]:
db_filename = 'my-budget-dev-v1.sqlite'

%run nuclear_option.py $db_filename
%run database_setup.py $db_filename

db = sql.connect(db_filename)

Creating an an empty database as "my-budget-dev-v1.sqlite"
Completed creating the database scheme. It looks as follows:


,name
0,money_pots
1,event_types
2,budget_pots
3,money_events
4,payments
5,recievings
6,transfers
7,budget_events
8,event_groups
9,event_in_group


The table money_pots: 


,key,description,liquid
0,KG,gemeinsames Konto,Yes
1,KE,Extrakonto zum gemeinsamen Konto,Yes
2,KM,Konto Max,Yes
3,KP,Konto Paul,Yes
4,KB,Konto Bundesbank,Yes
5,KC,Consorsbankkonto,No
6,BM,Bargeld Max,Yes
7,BP,Bargeld Paul,Yes
8,CB,Chipkarte Bundesbank,Semi
9,CT,Chipkarte Trianon,Semi


The table event_types: 


...

,group_id,event_id


The table database_event_types: 


,type,description
0,Erstellung,Erstellung eines Eintrages
1,Update,Hinzufügen von Information
2,Korrektur,Korrigieren eines Eintrages
3,Löschung,Löschen eines Eintrages


The table database_events: 


,id,type,date,description


In [3]:
excel_filename = 'GemeinsameBilanzierung_16_17__dev.xlsx'

current_sheet = 'August'
comment_sheet = 'august_'

# We take here the option data_only since for now we are not interested in the expression
wb = pyxl.load_workbook(excel_filename, data_only=True)


august = wb.get_sheet_by_name(current_sheet)
august_ = wb.copy_worksheet(august)
august_.title = comment_sheet
august_.sheet_state = 'hidden'

In [4]:
def list_from_range_string(range_string) :
    '''Extract all individual cell names from a excel range.
    
    Keyword arguments:
        range_string - The excel expression for the range
    
    Example:
        If range_string == 'A1:B3' then the list ['A1', 'B1', 'A2', 'B2', 'A3', 'B3'] is returned
    '''
    colon_position = range_string.find(':')
    if colon_position == -1 :
        raise
    first_cell = range_string[:colon_position]
    last_cell = range_string[colon_position+1:]
    
    first_row, first_col = a1_to_rowcol(first_cell)
    last_row, last_col = a1_to_rowcol(last_cell)
    
    return [rowcol_to_a1(i,j) for i,j in product(range(first_row, last_row+1), range(first_col, last_col+1))]

In [5]:
def generate_id(date) :
    '''Generate a new unique ID in the budgeter on the database. An ID is an integer with 8 digits, where 
    the first digit are based on the date and the last two digits are a serial number.
    
    Keyword arguments:
        date - the date on which the ID should be based
        
    Example: 
        If date==DateTime('2017-08-17') and the database contains the IDs 
        2017081701, 2017081702, 2017081703 and 2017081705, then the id 2017081704 is returned.
    
    Exceptions:
        IndexError - if all 99 possible serial numbers (01-99) have already been distributed.
        
        N.B. Could also Except, if the Database Call raises an exception.
    '''
    date_int = int(date.strftime('%Y%m%d00'))
    crsr = db.cursor()
    crsr.execute('SELECT id FROM money_events WHERE id BETWEEN {} AND {}'.format(date_int, date_int + 99))
    results = [row[0] for row in crsr.fetchall()]

    current_id = date_int + 1
    while current_id in results :
        current_id += 1
    if current_id > date_int + 99 :
        raise IndexError('Encountered to many ids for the date {}'.format(date))

    return current_id

In [6]:
def date_convert(item) :
    '''Try to get a DateTime from the excel cell, independent if it is represented by an integer, i.e. in 
    the native excel date format, or a string representation
    '''
    try :
          return pyxl.utils.datetime.from_excel(item)
    except :
        if type(item) is str :
            return pd.to_datetime(item)
        else :
            return pd.Timestamp(item)
        #pass
            

def get_df_by_range(sheet, first_cell, last_cell, date_cols=None) :
    '''Read a given range on the given sheet and return a DataFrame containing the data.
    
    Keyword arguments:
        sheet      - a openpyxl sheet object which is to be read
        first_cell - the top left cell of the range to be read; in excel cell notation
        last_cell  - the bottom right cell of the range to be read; in excel cell notation
        date_cols  - a column (or list of columns) which are assumed to contain dates and shall be returned
                     as pandas Timestamp object; can be given either as number (starting in zero) or a excel
                     column name
                     
        ToDo : Look if starting in zero is correct 
    '''
    data_rows = [[cell.value for cell in row] + ['{0}:{1}'.format(row[0].coordinate, row[-1].coordinate)]
        for row in august[first_cell:last_cell]]

    df = pd.DataFrame(data_rows)
    new_index = df.iloc[:,range(len(df.columns)-1)].dropna(how='all').index
    if date_cols is not None and type(date_cols) is int :
        df.iloc[:,date_cols] = df.iloc[:,date_cols].apply(date_convert).copy()
    elif type(date_cols) is str : 
        df.loc[:,date_cols] = df.loc[:,date_cols].apply(date_convert).copy()
    elif type(date_cols) is list :
        for col in date_cols :
            if type(col) is int :
                df.iloc[:,col] = df.iloc[:,col].apply(date_convert).copy()
            if type(col) is str : 
                df.loc[:,col] = df.loc[:,col].apply(date_convert).copy()
    return df.loc[new_index]

In [7]:
def put_comment_into_excel(sheet, cells, comment_text) :
    comment = pyxl.comments.Comment(comment_text, 'budgeter')
    if type(cells) is list :
        for cell in cells :
            try : 
                sheet[cell].comment = comment
            except :
                print(cell)
                print(comment)
                print(type(sheet))
    elif type(cells) is str :
        sheet[cells].comment = comment
    else :
        raise ValueError('Unexpected type of argument recieved for cells')

In [8]:
def put_payment_into_database_autogenerated(date, event_type, description, amount, 
                                            money_pot, budget_pot, comment) :
    the_id = generate_id(date)
    
    crsr = db.cursor()
    # ToDo : Hier machen wir die simplifizierende Annahme, dass alle Konto-Events Kartenzahlungen sind
    crsr.execute('''INSERT INTO money_events VALUES ({}, "{}", "{}", date("{}"));
        '''.format(the_id, event_type, description, date))
    crsr.execute('''INSERT INTO payments VALUES ({}, "{}", {}, NULL, NULL);
        '''.format(the_id,  money_pot, amount))
    crsr.execute('''INSERT INTO budget_events VALUES ({}, "{}", {}, NULL, NULL);
        '''.format(the_id, budget_pot, amount))
    crsr.execute('''INSERT INTO database_events VALUES ({}, "{}", "{}", "{}");
        '''.format(the_id, 'Erstellung', dt.now().strftime('%Y-%m-%d'), comment))
    db.commit()
    
    return the_id

def put_transfer_into_database_autogenerated(date, event_type, description, amount, 
                                             money_pot_source, money_pot_sink, 
                                             effect_date = None, comment = '') :
    the_id = generate_id(date)
    
    crsr = db.cursor()
    # ToDo : Hier machen wir die simplifizierende Annahme, dass alle Konto-Events Kartenzahlungen sind
    crsr.execute('''INSERT INTO money_events VALUES ({}, "{}", "{}", date("{}"));
        '''.format(the_id, event_type, description, date))
    crsr.execute('''INSERT INTO transfers VALUES ({}, "{}", "{}", {}, NULL, {});
        '''.format(the_id,  money_pot_source, money_pot_sink, amount,
                   'NULL' if effect_date is None else '"{}"'.format(effect_date)))
    crsr.execute('''INSERT INTO database_events VALUES ({}, "{}", "{}", "{}");
        '''.format(the_id, 'Erstellung', dt.now().strftime('%Y-%m-%d'), comment))
    db.commit()
    
    return the_id

In [9]:
def autogenerate_database_comment_multiple_excel_ranges(filename, ranges, comment_sheet) :
    try : 
        ranges_string = ', '.join(ranges[:-1]) + (' and ' if len(ranges) > 1 else '') + ranges[-1]
    except IndexError :
        raise ValueError('You should provide at least one Excel range')
    return 'This entry was automatically generated from the excel file ' + \
           '{}. It is based on the cells {}. A note has been '.format(
               filename, ranges_string) + \
           'added to the respective cells in the sheet {}.'.format(comment_sheet)
        
def autogenerate_database_comment(filename, range_, comment_sheet) :
    return autogenerate_database_comment_multiple_excel_ranges(filename, [range_], comment_sheet)

def autogenerate_excel_comment(date, db_filename, the_id) :
    return 'On {} this cell was automatically read and '.format(date) + \
           'inserted into the database {}. The id ofthe entry is {}.'.format(
               date, db_filename, the_id)

In [10]:
def automatically_enter_payment_row(date, payment_type, description, amount, money_pot, budget_type,
        ranges, excel_filename, current_sheet, comment_sheet, db_filename) :

    list_of_ranges = [current_sheet.title() + '!' + a_range for a_range in ranges]
    
    database_comment = autogenerate_database_comment_multiple_excel_ranges(
        excel_filename, list_of_ranges, comment_sheet)
    
    the_id = put_payment_into_database_autogenerated(date, payment_type, description, amount, 
                                                     money_pot, budget_type, database_comment)
   
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    
    #Get a list of all referenced cells in the ranges
    cells = list(chain.from_iterable([list_from_range_string(a_range) for a_range in ranges]))
    print(cells)
        
    put_comment_into_excel(comment_sheet, cells, excel_comment)
    
    return the_id

def automatically_enter_transfer_row(date, transfer_type, description, amount, money_pot_source, 
                                     money_pot_sink, ranges, excel_filename, current_sheet, 
                                     comment_sheet, db_filename, effect_date = None) :

    list_of_ranges = [current_sheet.title() + '!' + a_range for a_range in ranges]
    
    database_comment = autogenerate_database_comment_multiple_excel_ranges(
        excel_filename, list_of_ranges, comment_sheet)
    
    the_id = put_transfer_into_database_autogenerated(date, payment_type, description, amount, 
                                                      money_pot_source, money_pot_sink, database_comment)
   
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    
    #Get a list of all referenced cells in the ranges
    cells = list(chain.from_iterable([list_from_range_string(a_range) for a_range in ranges]))
        
    put_comment_into_excel(comment_sheet, cells, excel_comment)
    
    return the_id

In [11]:
def subset_data_frame(data_frame, rules) :
    ''' Subset a data_frame by a dictionary of rules. The rules should have the following form:
    {<column name> : <restriction>} where <restriction> is either a number, a string starting with 
    '<' or '>' and followed by number or a general string. If it is a simple string or number, the 
    subsetting expression
        data_frame[dataframe[<column name>] == <restriction>]
    will be used. The other case will be interpreted as
        data_frame[dataframe[<column name>] <|> <number>].
    '''
    
    result = data_frame
    for rule in rules.items() :
        column = rule[0]
        restriction = rule[1]
        
        if type(restriction) is str and restriction[0] in ['<', '>'] :
                try :
                    number = float(restriction[1:])
                except :
                    raise ValueError('After a rule starting with < or > you need to specify a number')
                result = result.loc[result[column] < number] if restriction[0] == '<' \
                            else result.loc[result[column] > number]
                continue

        result = result.loc[result[column] == restriction]
        if len(result) == 0 :
            print('After the rule ' + str(rule) + ' no item was left')
            return result
    
    print(str(len(result)) + ' items were selected.')
    return result

In [12]:
budgeting = get_df_by_range(august_, 'A6', 'D130', 2)
budgeting.columns = ['budget_type', 'description', 'date', 'amount', 'excel_range']

col_titles = ['description', 'date', 'amount', 'excel_range']

max_bargeld = get_df_by_range(august_, 'H7', 'J130', 1)
max_bargeld.columns = col_titles
max_bargeld['money_pot'] = 'BM'

paul_bargeld = get_df_by_range(august_, 'K7', 'M130', 1)
paul_bargeld.columns = col_titles
paul_bargeld['money_pot'] = 'BP'

konto = get_df_by_range(august_, 'N7', 'P130', 1)
konto.columns = col_titles
konto['money_pot'] = 'KG'

conjoined = pd.concat([max_bargeld, paul_bargeld, konto])

In [13]:
all_info = pd.merge(budgeting, conjoined, how='outer', 
                    on=['description', 'date', 'amount'], indicator=True)

all_info['treated'] = 'No'

In [17]:
display(all_info)

,budget_type,description,date,amount,excel_range_x,excel_range_y,money_pot,_merge,treated
0,R,Miete,2017-08-01,-568.000000,A6:D6,N13:P13,KG,both,Yes
1,R,Miete FFM,2017-08-02,-450.000000,A7:D7,N20:P20,KG,both,No
2,R,Berufsunfähigkeitsversicherung,2017-08-01,-49.050000,A8:D8,N12:P12,KG,both,No
3,R,Strom EnviaM,2017-08-30,-51.000000,A9:D9,N89:P89,KG,both,Yes
4,R,Vodafone,2017-08-28,-19.990000,A10:D10,NaN,NaN,left_only,No
5,M,Telefonie,2017-08-28,-6.200000,A11:D11,NaN,NaN,left_only,No
6,R,Rechtsschutzversicherung,2017-01-30,-13.900000,A12:D12,NaN,NaN,left_only,No
7,R,Haftpflichtversicherung,2017-08-01,-7.500000,A13:D13,N11:P11,KG,both,No
8,G,GEW,2017-07-05,-2.500000,A14:D14,NaN,NaN,left_only,No
9,R,Semestergebühr Paul,2016-06-21,-46.151667,A15:D15,NaN,NaN,left_only,No


In [16]:
miete_rules              = {'description' : 'Miete', 'amount' : -568,  'treated' : 'No'} 
haftpflicht_rules        = {'description' : 'Haftpflichtversicherung', 'amount' : -7.5,  'treated' : 'No'} 
berufsunfaehigkeit_rules = {'description' : 'Berufsunfähigkeitsversicherung', 'amount' : -49.05,  'treated' : 'No'} 
strom_rules_rules        = {'description' : 'Strom EnviaM', 'amount' : -51,  'treated' : 'No'} 
vodafone_rules           = {'description' : 'Vodafone', 'amount' : -19.99,  'treated' : 'No'} 
handy_paul_rules         = {'description' : 'Handy Paul', 'amount' : -7.99,  'treated' : 'No'} 
handy_max_rules          = {'description' : 'Handy Max', 'amount' : -7.99,  'treated' : 'No'} 
music_paul_rules         = {'description' : 'Apple Music Paul', 'amount' : -4.99,  'treated' : 'No'} 
music_max_rules          = {'description' : 'Spotify Max', 'amount' : -4.99,  'treated' : 'No'} 
backblaze_paul_rules     = {'description' : 'Backblaze Paul', 'amount' : -3.77,  'treated' : 'No'} 
backblaze_max_rules      = {'description' : 'Backblaze Max', 'amount' : -3.77,  'treated' : 'No'} 
semester_paul_rules      = {'description' : 'Semestergebühr Paul', 'amount' : -46.15,  'treated' : 'No'} 
semester_max_rules       = {'description' : 'Semestergebühr Max', 'amount' : -50.57,  'treated' : 'No'} 
sport_jahre_rules        = {'description' : '', 'amount' : -8,  'treated' : 'No'} 
sport_rules              = {'description' : 'Fitnessstudio', 'amount' : -39.8,  'treated' : 'No'} 
gew_rules                = {'description' : 'GEW', 'amount' : -2.5,  'treated' : 'No'} 
gez_rules                = {'description' : 'GEZ', 'amount' : -17.5,  'treated' : 'No'} 

all_rules = { 1 : [miete_rules              ],
              2 : [haftpflicht_rules        ],
              3 : [berufsunfaehigkeit_rules ],
              4 : [strom_rules_rules        ],
              5 : [vodafone_rules           ],
              6 : [handy_paul_rules         ],
              7 : [handy_max_rules          ],
              8 : [music_paul_rules         ],
              9 : [music_max_rules          ],
             10 : [backblaze_paul_rules, backblaze_max_rules],
             11 : [semester_paul_rules      ],
             12 : [semester_max_rules       ],
             13 : [sport_jahre_rules        ],
             14 : [sport_rules              ],
             15 : [gew_rules                ],
             16 : [gez_rules                ]}

monthly = [1, 4, 8, 9, 14]
bigger_intervalls = [10, 11, 12, 13, 15, 16]
variable_prices = [5, 6, 7]

for rule_number in monthly :
    rule = all_rules[rule_number][0]
    
    the_info = subset_data_frame(all_info, rule)

    for index, row in the_info.iterrows() :
        the_id = automatically_enter_payment_row(row['date'], 'Dauerauftrag', 
            row['description'], row['amount'], row['money_pot'], 
            row['budget_type'], [row['excel_range_x'], row['excel_range_y']],
            excel_filename, current_sheet, comment_sheet, db_filename)

        crsr = db.cursor()
        crsr.execute('''INSERT INTO event_in_group VALUES (1, {});'''.format(the_id))
        db.commit()

        all_info.loc[index, 'treated'] = 'Yes'

1 items were selected.
['A6', 'B6', 'C6', 'D6', 'N13', 'O13', 'P13']
A6
Comment: On 2017-09-22 this cell was automatically read and inserted into the database 2017-09-22. The id ofthe entry is my-budget-dev-v1.sqlite. by budgeter
<class 'str'>
B6
Comment: On 2017-09-22 this cell was automatically read and inserted into the database 2017-09-22. The id ofthe entry is my-budget-dev-v1.sqlite. by budgeter
<class 'str'>
C6
Comment: On 2017-09-22 this cell was automatically read and inserted into the database 2017-09-22. The id ofthe entry is my-budget-dev-v1.sqlite. by budgeter
<class 'str'>
D6
Comment: On 2017-09-22 this cell was automatically read and inserted into the database 2017-09-22. The id ofthe entry is my-budget-dev-v1.sqlite. by budgeter
<class 'str'>
N13
Comment: On 2017-09-22 this cell was automatically read and inserted into the database 2017-09-22. The id ofthe entry is my-budget-dev-v1.sqlite. by budgeter
<class 'str'>
O13
Comment: On 2017-09-22 this cell was automatically 

In [ ]:
## Putting all the Info which had matching info on the left and the right hand of the budget sheet into
## the database.

## Only doing the "Barzahlung" payments right now

for BB in ['BM', 'BP'] :
    subsetting_rules = {'_merge' : 'both', 'money_pot' : BB,  'treated' : 'No'}

    full_match_bargeld = subset_data_frame(all_info, subsetting_rules)

    for index, row in full_match_bargeld.iterrows() :
        the_id = automatically_enter_payment_row(row['date'], 'Barzahlung', 
            row['description'], row['amount'], row['money_pot'], 
            row['budget_type'], [row['excel_range_x'], row['excel_range_y']],
            excel_filename, current_sheet, comment_sheet, db_filename)

        
        all_info.loc[index, 'treated'] = 'Yes' 

In [ ]:
## For the "GK" (Gemeinsames Konto) Payments we create an excel sheet, where we can change the payment type
subsetting_rules = {'_merge' : 'both', 'money_pot' : 'KG',  'treated' : 'No'}
full_match_konto = subset_data_frame(all_info, subsetting_rules)
full_match_konto['payment_type'] = 'Kartenzahlung'
full_match_konto['use_data'] = 'X'

In [ ]:
gk_wb_filename = 'testtest.xlsx'
full_match_konto.to_excel(gk_wb_filename)
gk_wb = pyxl.load_workbook(gk_wb_filename, data_only=True)

gk_sheet = gk_wb.active

the_fill = pyxl.styles.PatternFill(fill_type='solid', start_color='FFFF02')
        
the_list_a = ['Barzahlung', 'Kartenzahlung', 'Überweisung', 'Dauerauftrag',
            'SEPA-Mandat', 'Bankeinzug', 'Abheben', 'Kontotransfer', 
            'Bargeldtransfer', 'Einnahme', 'Geldfund']
the_list_b = ['B', 'K', 'U', 'D', 'S', 'BE', 'A', 'KT', 'BT', 'E', 'GF']

the_list_str = '"' + ','.join(the_list_a + the_list_b) + '"'

dv1 = pyxl.worksheet.datavalidation.DataValidation(type="list", formula1=the_list_str, allow_blank=False)
dv1.error ='Your entry is not in the list'
dv1.errorTitle = 'Invalid Entry'
dv1.prompt = 'Please select from the list'
dv1.promptTitle = 'List Selection'

dv2 = pyxl.worksheet.datavalidation.DataValidation(type="list", formula1='"X,x"', allow_blank=True)
dv2.error ='Your entry is not in the list'
dv2.errorTitle = 'Invalid Entry'
dv2.prompt = 'Please check with an X or leave blank'
dv2.promptTitle = 'Checkmark'

dv1.ranges.append('K2:K' + str(gk_sheet.max_row))
dv2.ranges.append('L2:L' + str(gk_sheet.max_row))

gk_sheet.add_data_validation(dv1)
gk_sheet.add_data_validation(dv2)

for i in range(2, gk_sheet.max_row + 1) :
    gk_sheet['D' + str(i)].number_format = 'DD/MM/YY'
    gk_sheet['K' + str(i)].fill = the_fill
    gk_sheet['L' + str(i)].fill = the_fill
    
for column_cells in gk_sheet.columns:
    length = max(len(str(cell.value)) for cell in column_cells)
    gk_sheet.column_dimensions[column_cells[0].column].width = length
    
for row in ['F', 'G', 'H', 'I', 'J'] :
    gk_sheet.column_dimensions[row].hidden = True    
    
gk_wb.save(gk_wb_filename)

In [ ]:
open_excel_file(gk_wb_filename)
        
checked_full_match_konto = parse_excel_to_pandas(gk_wb_filename)

In [ ]:
for index, row in checked_full_match_konto.iterrows() :
    if checked_full_match_konto.loc[index, 'use_data'] is np.nan :
        continue
        
    the_id = automatically_enter_payment_row(row['date'], row['payment_type'], 
            row['description'], row['amount'], row['money_pot'], 
            row['budget_type'], [row['excel_range_x'], row['excel_range_y']],
            excel_filename, current_sheet, comment_sheet, db_filename)
    
    all_info.loc[index, 'treated'] = 'Yes'

In [ ]:
## List of all fully matched things we didnt treat yet
subsetting_rules = {'_merge' : 'both', 'treated' : 'No'}
remaining_matched = subset_data_frame(all_info, subsetting_rules)

display(remaining_matched)

In [ ]:
display(all_info[all_info['description'].str.contains('Schatulle')])

subsetting_rules = {'treated' : 'xxx'}
schatulle_info = subset_data_frame(all_info, subsetting_rules)

for direction in ['an', 'aus'] :
    descriptor = 'Geld ' + direction + ' Schatulle'
    subsetting_rules = {'description' : descriptor, 'treated' : 'No'}
    schatulle_info = schatulle_info.append(subset_data_frame(all_info, subsetting_rules))
    
display(schatulle_info)

In [ ]:
for index, row in schatulle_info.iterrows() :
    i = 0
    while True :
        the_date = all_info.loc[index+i, 'date']
        if str(the_date) == 'NaT' :
            the_date = all_info.loc[index-i, 'date']
            if str(the_date) == 'NaT' :
                i = i + 1
                continue
        
        break
            
    the_id = put_transfer_into_database_autogenerated(
        the_date, 'Bargeldtransger', row['description'], row['amount'], 
        row['money_pot'], 'SB', database_comment)
    
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    cells = list_from_range_string(row['excel_range_y'])
        
    put_comment_into_excel(august_, cells, excel_comment)
    
    all_info.loc[index, 'treated'] = 'Yes'

In [ ]:
## The Three individual usages of the White Trianon Card
subsetting_rules = {'description' : 'Geld aufladen Trianon', 'amount' : -20, 'treated' : 'No'}
trianon_aufladen = subset_data_frame(all_info, subsetting_rules)

for index, row in trianon_aufladen.iterrows() :
    database_comment = autogenerate_database_comment(
        excel_filename, current_sheet + '!' + row['excel_range_y'], comment_sheet)
    
    the_id = put_transfer_into_database_autogenerated(row['date'], 'Aufladen',
                 row['description'], row['amount'], row['money_pot'], 'CT', 
                 database_comment)
   
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    cells = list_from_range_string(row['excel_range_y'])
        
    put_comment_into_excel(august_, cells, excel_comment)
    
    all_info.loc[index, 'treated'] = 'Yes'


subsetting_rules = {'budget_type' : 'AM', 'description' : 'Doppelter Kaffee Trianon', 
                    'amount' : -1.5, 'treated' : 'No'}
trianon_doppelter_kaffee = subset_data_frame(all_info, subsetting_rules)

for index, row in trianon_doppelter_kaffee.iterrows() :
    database_comment = autogenerate_database_comment(
        excel_filename, current_sheet + '!' + row['excel_range_x'], comment_sheet)
    
    the_id = put_payment_into_database_autogenerated(row['date'], 'Kartenzahlung',
                 row['description'], row['amount'], 'CT', 
                 row['budget_type'], database_comment)
   
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    cells = list_from_range_string(row['excel_range_x'])
        
    put_comment_into_excel(august_, cells, excel_comment)
    
    all_info.loc[index, 'treated'] = 'Yes'

subsetting_rules = {'budget_type' : 'AM', 'description' : 'Kaffee', 
                    'amount' : -0.75, 'treated' : 'No'}
trianon_einfacher_kaffee = subset_data_frame(all_info, subsetting_rules)

for index, row in trianon_einfacher_kaffee.iterrows() :
    database_comment = autogenerate_database_comment(
        excel_filename, current_sheet + '!' + row['excel_range_x'], comment_sheet)
    
    the_id = put_payment_into_database_autogenerated(row['date'], 'Kartenzahlung',
                 row['description'], row['amount'], 'CT', 
                 row['budget_type'], database_comment)
   
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    cells = list_from_range_string(row['excel_range_x'])
        
    put_comment_into_excel(august_, cells, excel_comment)
    
    all_info.loc[index, 'treated'] = 'Yes'

In [ ]:
subsetting_rules = {'budget_type' : 'AM', 'treated' : 'No'}
bundesbank_karte = subset_data_frame(all_info, subsetting_rules)

for index, row in bundesbank_karte.iterrows() :
    database_comment = autogenerate_database_comment(
        excel_filename, current_sheet + '!' + row['excel_range_x'], comment_sheet)
    
    the_id = put_payment_into_database_autogenerated(row['date'], 'Kartenzahlung',
                 row['description'], row['amount'], 'CB', 
                 row['budget_type'], database_comment)
   
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    cells = list_from_range_string(row['excel_range_x'])
        
    put_comment_into_excel(august_, cells, excel_comment)
    
    all_info.loc[index, 'treated'] = 'Yes'

In [ ]:
subsetting_rules = {'description' : 'Geld abheben', '_merge' : 'right_only', 'treated' : 'No'}
geld_abheben = subset_data_frame(all_info, subsetting_rules)

subsetting_rules = {'description' : 'Geld abheben', 'amount' : '>0', '_merge' : 'right_only', 'treated' : 'No'}
geld_abheben_gotten = subset_data_frame(all_info, subsetting_rules)

subsetting_rules = {'description' : 'Geld abheben', 'amount' : '<0', '_merge' : 'right_only', 'treated' : 'No'}
geld_abheben_drawn = subset_data_frame(all_info, subsetting_rules)
geld_abheben_drawn['amount_abs'] = - geld_abheben_drawn['amount']

## Subselect relevant columns
abheben_gotten = geld_abheben_gotten[['date', 'amount', 'money_pot', 'description', 
                                      'excel_range_y']].reset_index(level=0)
abheben_drawn = geld_abheben_drawn[['date', 'amount_abs', 'money_pot', 
                                    'excel_range_y']].reset_index(level=0)


abheben_data = pd.merge(abheben_gotten, abheben_drawn, how='outer', left_on = ['date', 'amount'], 
                        right_on=['date', 'amount_abs'], left_index=True, indicator=True)

display(abheben_data)


for index, row in abheben_data[abheben_data['_merge'] == 'both'].iterrows() :
    database_comment = autogenerate_database_comment_multiple_excel_ranges(
        excel_filename, [current_sheet + '!' + row['excel_range_y_x'],
        current_sheet + '!' + row['excel_range_y_y']],comment_sheet)
            
    the_id = put_transfer_into_database_autogenerated(
        row['date'], 'Abheben', row['description'], row['amount'], 
        row['money_pot_y'], row['money_pot_x'], database_comment)
    
    excel_comment = autogenerate_excel_comment(
        dt.now().strftime('%Y-%m-%d'), db_filename, the_id)
    cells = list_from_range_string(row['excel_range_y_x']) + \
            list_from_range_string(row['excel_range_y_y'])
        
    put_comment_into_excel(august_, cells, excel_comment)
    
    all_info.loc[row['index_x'], 'treated'] = 'Yes'
    all_info.loc[row['index_y'], 'treated'] = 'Yes'

In [ ]:
## For the "GK" (Gemeinsames Konto) Payments we create an excel sheet, where we can change the payment type
subsetting_rules = {'_merge' : 'right_only', 'treated' : 'No'}
remaining = subset_data_frame(all_info, subsetting_rules)
remaining['is_transfer'] = ''
remaining['money_pot_source'] = remaining['amount'].apply(lambda x : 1 if x < 0 else 0) * \
                                remaining['money_pot'] 
remaining['money_pot_sink'] = remaining['amount'].apply(lambda x : 1 if x > 0 else 0) * \
                              remaining['money_pot']
display(remaining)

In [ ]:
mark_transfers_filename = 'testtest2.xlsx'
remaining.to_excel(mark_transfers_filename)
mark_transfers = pyxl.load_workbook(mark_transfers_filename, data_only=True)

remaining_sheet = mark_transfers.active

the_fill = pyxl.styles.PatternFill(fill_type='solid', start_color='FFFF02')

dv1 = pyxl.worksheet.datavalidation.DataValidation(type="list", formula1='"X,x"', allow_blank=True)
dv1.error ='Your entry is not in the list'
dv1.errorTitle = 'Invalid Entry'
dv1.prompt = 'Please check with an X or leave blank'
dv1.promptTitle = 'Checkmark'

list_of_accounts = ['KG', 'KE', 'KM', 'KP', 'KB', 'KC', 'BM', 'BP', 'CB', 'CT', 'CM', 'GM', 'SB', 'SF']
rule = '"' + ','.join(list_of_accounts) + '"'
dv2 = pyxl.worksheet.datavalidation.DataValidation(type="list", formula1=rule, allow_blank=True)
dv2.error ='Your entry is not in the list'
dv2.errorTitle = 'Invalid Entry'
dv2.prompt = 'Please select from the list'
dv2.promptTitle = 'List Selection'

dv1.ranges.append('K2:K' + str(remaining_sheet.max_row))
dv2.ranges.append('L2:M' + str(remaining_sheet.max_row))

remaining_sheet.add_data_validation(dv1)
remaining_sheet.add_data_validation(dv2)

for i in range(2, remaining_sheet.max_row + 1) :
    remaining_sheet['D' + str(i)].number_format = 'DD/MM/YY'
    remaining_sheet['K' + str(i)].fill = the_fill
    remaining_sheet['L' + str(i)].fill = the_fill
    remaining_sheet['M' + str(i)].fill = the_fill
    
for column_cells in remaining_sheet.columns:
    length = max(len(str(cell.value)) for cell in column_cells)
    remaining_sheet.column_dimensions[column_cells[0].column].width = length
    
for row in ['F', 'G', 'H', 'I', 'J'] :
    remaining_sheet.column_dimensions[row].hidden = True    
    
mark_transfers.save(mark_transfers_filename)

In [ ]:
input('Make any input to continue with opening the Excel sheet.')
os.system('open ' + mark_transfers_filename)


data_is_final = False

while not data_is_final :
    read_data = pd.read_excel(gk_wb_filename)
    
    ## Here one could do a replace with a dict if the transaction type was abbreviated

    print('The read data is:')
    display(read_data)
    print('')

    request = input('If this is not the intended data, enter any number:')

    if not is_number(request) :
        print('OK.')
        data_is_final = True
    else : 
        os.system('open ' + temp_name)
        input('Make any input to continue.')
        
checked_full_match_konto = read_data

In [ ]:
remaining = all_info[all_info['treated'] == 'No']

display(remaining)

print("number remaining: " + str(len(remaining)))

In [ ]:
for table in ["money_events", "payments", "budget_events"] :
    print('The table {}: '.format(table))
    display(pd.read_sql_query('SELECT * FROM {};'.format(table), db))

In [ ]:
wb.save('GemBil.xlsx')

In [ ]:
db.close()